In [1]:
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dot, Flatten
from sklearn.model_selection import train_test_split

In [2]:
data_project = pd.read_csv('generate_random_data_project.csv',names=["id_project", "game_type"])
freelancer_preference = pd.read_csv('new_combined_preferences.csv')

In [3]:
result_data = []

# Loop melalui setiap baris di freelancer_preference
for _, row in freelancer_preference.iterrows():
    id_freelancer = row['id_freelancer']
    
    # Loop melalui setiap kolom kecuali 'id_freelancer'
    for col in freelancer_preference.columns[1:]:
        game_type = col
        preference_freelancer = row[col]
        
        # Filter data_project berdasarkan game_type
        filtered_data_project = data_project[data_project['game_type'] == game_type]
        
        # Loop melalui setiap baris di data_project yang sesuai
        for _, project_row in filtered_data_project.iterrows():
            id_project = project_row['id_project']
            
            # Tambahkan data ke list
            result_data.append({'id_project': id_project, 'id_freelancer': id_freelancer, 'game_type': game_type, 'preference_freelancer': preference_freelancer})

In [4]:
result = pd.DataFrame(result_data)

In [5]:
print(result)

      id_project  id_freelancer  game_type  preference_freelancer
0              1              1       Moba                      2
1             14              1       Moba                      2
2             21              1       Moba                      2
3             31              1       Moba                      2
4             53              1       Moba                      2
...          ...            ...        ...                    ...
89595       1247             70  Adventure                      5
89596       1254             70  Adventure                      5
89597       1256             70  Adventure                      5
89598       1262             70  Adventure                      5
89599       1274             70  Adventure                      5

[89600 rows x 4 columns]


In [6]:
# result.to_csv('new_merged_data.csv', index=False)

In [7]:
projects = pd.read_csv('generate_random_data_project.csv')
preferences = pd.read_csv('new_merged_data.csv')

In [8]:
n_project = preferences.id_project.nunique()
n_freelancer = preferences.id_freelancer.nunique()

In [9]:
train, test = train_test_split(preferences, test_size=0.2)

In [10]:
EMBEDDING_DIM = 30

#input layer
project_input = Input(shape=1)
freelancer_input = Input(shape=1)

#embedding layer
project_embedding = Embedding(n_project+1, EMBEDDING_DIM)(project_input)
freelancer_embedding = Embedding(n_freelancer+1, EMBEDDING_DIM)(freelancer_input)

#flatten layer
project_flat = Flatten()(project_embedding)
freelancer_flat = Flatten()(freelancer_embedding)

#output layer
output = Dot(1)([project_flat, freelancer_flat])

model = Model([project_input, freelancer_input],[output])

In [11]:
model.compile(optimizer='adam', loss='mse')

In [12]:
model.fit(x=[train.id_project, train.id_freelancer], y=[train.preference_freelancer],
         epochs=30, batch_size=128)

Epoch 1/30
560/560 [==============================] - 3s 4ms/step - loss: 11.8990
Epoch 2/30
560/560 [==============================] - 2s 4ms/step - loss: 1.5575
Epoch 3/30
560/560 [==============================] - 2s 4ms/step - loss: 1.2766
Epoch 4/30
560/560 [==============================] - 2s 4ms/step - loss: 1.2722
Epoch 5/30
560/560 [==============================] - 2s 4ms/step - loss: 1.2495
Epoch 6/30
560/560 [==============================] - 2s 3ms/step - loss: 1.1729
Epoch 7/30
560/560 [==============================] - 2s 3ms/step - loss: 1.0684
Epoch 8/30
560/560 [==============================] - 2s 3ms/step - loss: 0.9786
Epoch 9/30
560/560 [==============================] - 2s 3ms/step - loss: 0.8932
Epoch 10/30
560/560 [==============================] - 2s 4ms/step - loss: 0.7837
Epoch 11/30
560/560 [==============================] - 2s 3ms/step - loss: 0.6419
Epoch 12/30
560/560 [==============================] - 2s 3ms/step - loss: 0.4852
Epoch 13/30
560/560 [===

In [13]:
model.evaluate(x=[test.id_project, test.id_freelancer], y=[test.preference_freelancer])

560/560 [==============================] - 2s 3ms/step - loss: 0.0017


0.0016578680370002985

In [14]:
predictions = model.predict([test.id_project, test.id_freelancer])

560/560 [==============================] - 2s 3ms/step


In [15]:
for i in range(10):
    print(f"Actual Rating: {test.preference_freelancer.iloc[i]}, Predicted Rating: {predictions[i][0]}")

Actual Rating: 5, Predicted Rating: 4.967432022094727
Actual Rating: 3, Predicted Rating: 2.9577767848968506
Actual Rating: 3, Predicted Rating: 3.1345834732055664
Actual Rating: 4, Predicted Rating: 3.9838085174560547
Actual Rating: 2, Predicted Rating: 2.0051963329315186
Actual Rating: 5, Predicted Rating: 5.012701511383057
Actual Rating: 3, Predicted Rating: 2.9991443157196045
Actual Rating: 4, Predicted Rating: 3.9565746784210205
Actual Rating: 2, Predicted Rating: 2.0089874267578125
Actual Rating: 4, Predicted Rating: 3.923888683319092


In [22]:
# import numpy as np
# import matplotlib.pyplot as plt

# def get_recommendations(id_freelancer, projects, model):
#     projects = projects.copy()
#     id_freelancers = np.array([id_freelancer] * len(projects))
#     results = model([projects.id_project.values, id_freelancers]).numpy().reshape(-1)

#     projects['predicted_rating'] = pd.Series(results)
#     projects = projects.sort_values('predicted_rating', ascending=False)

#     print(f'Recommendations for user {id_freelancer}')
#     return(projects)

# get_recommendations(100, projects, model)

In [43]:
import numpy as np
import tensorflow as tf
import pandas as pd

def get_recommendations(id_freelancer, projects, model):
    try:
        projects = projects.copy()
        id_freelancers = np.array([id_freelancer] * len(projects))
        results = model([projects.id_project.values, id_freelancers]).numpy().reshape(-1)

        projects['predicted_rating'] = pd.Series(results)
        projects = projects.sort_values('predicted_rating', ascending=False)

        print(f'Recommendations for user {id_freelancer}')
        return projects
    except tf.errors.InvalidArgumentError:
        print(f'User {id_freelancer} not found. Returning random recommendations.')
        # Menggunakan np.random.randint untuk mendapatkan nilai random_state yang berbeda setiap kali dijalankan
        random_state = np.random.randint(1, 100)
        projects = projects.sample(n=10, random_state=random_state)
        return projects

# Contoh pemanggilan fungsi
get_recommendations(777, projects, model)


User 777 not found. Returning random recommendations.


,id_project,game_type
522,523,Sports
256,257,RPG
18,19,Horor
818,819,FPS
965,966,Open World
181,182,Horor
830,831,Battle Royal
553,554,Puzzle
762,763,RPG
1234,1235,Moba


In [19]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 1)]                  0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 1)]                  0         []                            
                                                                                                  
 embedding (Embedding)       (None, 1, 30)                38430     ['input_1[0][0]']             
                                                                                                  
 embedding_1 (Embedding)     (None, 1, 30)                2130      ['input_2[0][0]']             
                                                                                              

In [20]:
model.save('recommender_model.h5')

c:\users\crozi\appdata\local\programs\python\python39\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
